# Notebook TFM

## Carga de datos

In [ ]:
import pandas as pd
import os
import matplotlib as plt
import numpy as np

In [ ]:
dir = './Hangzhou-mobility-data-set'
archivos_csv = [archivo for archivo in os.listdir(dir) if archivo.startswith('record') and archivo.endswith('.csv')]
dataframes = [pd.read_csv(os.path.join(dir, archivo)) for archivo in archivos_csv]
records = pd.concat(dataframes, ignore_index=True)
est = pd.read_csv('./Hangzhou-mobility-data-set/Metro_roadMap.csv')
meteo = pd.read_csv('./Hangzhou-mobility-data-set/HangzhouMeteoData.csv')

In [ ]:
records['time'] = pd.to_datetime(records['time'])

In [ ]:
records

In [ ]:
est

In [ ]:
meteo

In [ ]:
meteo.columns

In [ ]:
lala = meteo[(meteo['icon'].notna()) & meteo['icon']!=0]
lala['icon']

## Comprobación de nulos y tipos

In [ ]:
cantidad_de_ceros = (meteo['precip'] == 0).sum()
cantidad_de_ceros

In [ ]:
#Valores nulos en los records

hay_nan = records.isna().any().any()
if hay_nan:
    print("El DataFrame contiene valores NaN.")
else:
    print("El DataFrame no contiene valores NaN.")

In [ ]:
#Valores nulos en las estaciones
hay_nan = est.isna().any().any()
if hay_nan:
    print("El DataFrame contiene valores NaN.")
else:
    print("El DataFrame no contiene valores NaN.")

In [ ]:
#Valores nulos en las estaciones
hay_nan = meteo.isna().any().any()
if hay_nan:
    print("El DataFrame contiene valores NaN.")
else:
    print("El DataFrame no contiene valores NaN.")

In [ ]:
# Filtrar las columnas con valores nulos
columnas_con_nulos = meteo.columns[meteo.isnull().any()]

# Mostrar las columnas con valores nulos y la cantidad de valores nulos en cada una
for columna in columnas_con_nulos:
    cantidad_nulos = meteo[columna].isnull().sum()
    print(f'Columna "{columna}" tiene {cantidad_nulos} valores nulos.')

In [ ]:
records.dtypes

In [ ]:
records.dtypes

In [ ]:
records

## Exploración de los datos

### Horas no completas

In [ ]:
# Supongamos que tienes un DataFrame llamado records
# Crea un DataFrame con las columnas 'time', 'stationID' y 'day'
df_time_station_day = records[['time', 'stationID']]
df_time_station_day['day'] = df_time_station_day['time'].dt.date

# Extrae la hora de la columna 'time'
df_time_station_day['hour'] = df_time_station_day['time'].dt.hour

In [ ]:
# Agrupa por día, hora y estación y cuenta las observaciones en cada grupo
daily_hourly_station_counts = df_time_station_day.groupby(['day', 'hour', 'stationID']).size().reset_index(name='count')
daily_hourly_station_counts

In [ ]:
# Obtiene la lista de todas las estaciones únicas
estaciones_unicas = df_time_station_day['stationID'].unique()
estaciones_unicas.sort()
estaciones_unicas

In [ ]:
hours = np.arange(24)
hours

In [ ]:
daily_hourly_station_counts[(daily_hourly_station_counts['stationID'] == 4) & (daily_hourly_station_counts['hour'] == 16)].count()

In [ ]:
#for hour in range(24):
hours_to_delete = []
est_and_hour = []
for hour in hours:
    for est in estaciones_unicas:
        if (daily_hourly_station_counts[(daily_hourly_station_counts['stationID'] == est) & (daily_hourly_station_counts['hour'] == hour)].count().iloc[0] != 25):# & hora not in hours_to_delete:
            hours_to_delete.append(hour)
            est_and_hour.append([est,hour])

In [ ]:
len(est_and_hour)

In [ ]:
hours_to_delete.sort()
unique_values = list(set(hours_to_delete))

# Print the unique values
print(unique_values)

In [ ]:
records = records.loc[~records['time'].dt.hour.isin(unique_values)]
records

### Cálculo correlación

In [ ]:
df_agrupado = records.set_index('time').drop(columns=['payType']).groupby([pd.Grouper(freq='H', level=0),'stationID', 'lineID','status'])['userID'].agg('count').reset_index()
df_agrupado

In [ ]:
# Definir un diccionario de mapeo de valores únicos en "lineID" a números
mapeo_lineID = {
    'A': 1,
    'B': 2,
    'C': 3,
    # Agrega más mapeos según sea necesario
}

# Aplicar la conversión utilizando el método map
df_agrupado['lineID'] = df_agrupado['lineID'].map(mapeo_lineID)
df_agrupado['lineID'] = df_agrupado['lineID'].astype('category')
df_agrupado.corr()

In [ ]:
df_agrupado.corr()['userID']

In [ ]:
test_corr = df_agrupado[(df_agrupado['stationID']==10) & (df_agrupado['time'].dt.day==20)]
test_corr.corr()

In [ ]:
test_corr

In [ ]:
# Agrupar por la columna 'stationID' y contar los valores únicos en 'lineID'
stations_with_different_lines = df_agrupado.groupby('stationID')['lineID'].nunique()

# Filtrar las estaciones que tienen más de un valor único en 'lineID'
stations_with_multiple_lines = stations_with_different_lines[stations_with_different_lines > 1]
stations_with_multiple_lines

### Valores atípicos

In [ ]:
df_agrupado = records.set_index('time')
df_agrupado = df_agrupado.groupby(['stationID', df_agrupado.index.date, 'status'])['userID'].count().unstack(fill_value=0)
df_agrupado.rename(columns={0: 'salida', 1: 'entrada'}, inplace=True)
df_agrupado

In [ ]:
df_agrupado.reset_index(inplace=True)
df_agrupado.rename(columns={"level_1": 'time'}, inplace=True)

In [ ]:
df_agrupado

In [ ]:
filas_estacion  = df_agrupado[df_agrupado['stationID'] == 15]
filas_estacion 

In [ ]:
df_agrupado['entrada'].plot()

In [ ]:
excepciones_entrada = df_agrupado[df_agrupado['entrada']>200000]
excepciones_entrada

In [ ]:
df_agrupado['salida'].plot()

In [ ]:
excepciones_entrada = df_agrupado[df_agrupado['salida']>150000]
excepciones_entrada

### Agrupaciones varias

In [ ]:
df_agrupado = records.set_index('time')
df_agrupado = df_agrupado.groupby([pd.Grouper(freq='H', level=0),'stationID', 'lineID','payType','status'])['userID'].agg('count').reset_index()
df_agrupado

In [ ]:
df_agrupado = records.set_index('time')

In [ ]:
df_agrupado = df_agrupado.groupby(['stationID', df_agrupado.index.date, 'status'])['userID'].count().unstack(fill_value=0)
df_agrupado.rename(columns={0: 'salida', 1: 'entrada'}, inplace=True)

In [ ]:
records

In [ ]:
'''records.set_index('time', inplace=True)
df_agrupado = records.set_index('time').groupby('stationID')
for estacion, datos in df_agrupado:
    print(f"Estación {estacion}:")
    print(datos)'''

In [ ]:
'''resultados = pd.DataFrame()
for estacion, datos in df_agrupado:
    recuento_usuarios = datos['userID'].resample('10T').count()
    # Agregar los resultados al DataFrame de resultados
    resultados[f'Estacion_{estacion}'] = recuento_usuarios

resultados'''

In [ ]:
df_agrupado

In [ ]:
df_agrupado.reset_index(inplace=True)
df_agrupado

In [ ]:
#test_in_out = records.groupby(['stationID', records.index.date, records.index.hour, 'status'])['userID'].count().unstack(fill_value=0)
df_agrupado = records.set_index('time')
df_agrupado = df_agrupado.groupby(['stationID', df_agrupado.index, 'status'])['userID'].count().unstack(fill_value=0)
df_agrupado.rename(columns={0: 'salida', 1: 'entrada'}, inplace=True)
df_agrupado

In [ ]:
df_agrupado = records.set_index('time')
df_agrupado = df_agrupado.groupby(['stationID', pd.Grouper(freq='H', level=0), 'status'])['userID'].count().unstack(fill_value=0)
df_agrupado.rename(columns={0: 'salida', 1: 'entrada'}, inplace=True)
df_agrupado

In [ ]:
df_agrupado = records.set_index('time').drop(columns=['payType']).groupby([pd.Grouper(freq='H', level=0),'stationID', 'lineID','status'])['userID'].agg('count').reset_index()
# Definir un diccionario de mapeo de valores únicos en "lineID" a números
mapeo_lineID = {
    'A': 1,
    'B': 2,
    'C': 3,
    # Agrega más mapeos según sea necesario
}

# Aplicar la conversión utilizando el método map
df_agrupado['lineID'] = df_agrupado['lineID'].map(mapeo_lineID)
df_agrupado['lineID'] = df_agrupado['lineID'].astype('category')
df_agrupado

In [ ]:
df_agrupado = records.set_index('time')
#Para agrupar por horas poner 'H' en la frecuencia del Grouper
df_agrupado = df_agrupado.groupby(['stationID', pd.Grouper(freq='10T', level=0),'lineID' ,'status'])['userID'].count().unstack(fill_value=0)
df_agrupado.rename(columns={0: 'salida', 1: 'entrada'}, inplace=True)
df_agrupado.reset_index(inplace=True)
mapeo_lineID = {
    'A': 1,
    'B': 2,
    'C': 3,
    # Agrega más mapeos según sea necesario
}

# Aplicar la conversión utilizando el método map
df_agrupado['lineID'] = df_agrupado['lineID'].map(mapeo_lineID)
df_agrupado['lineID'] = df_agrupado['lineID'].astype('category')

In [ ]:
df_agrupado.corr()

In [ ]:
test_corr = df_agrupado[(df_agrupado['time'].dt.day==20)]
test_corr.corr()

In [ ]:
df_agrupado

In [ ]:
df_agrupado.dtypes

### Operaciones con dataset METEO

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
meteo.rename(columns={"datetime": 'time'}, inplace=True)
meteo['time'] = pd.to_datetime(meteo['time'])

In [ ]:
meteo.dtypes

In [ ]:
"""
# Contar valores diferentes de 0 en la columna
condicion_1 = (meteo['precip'] != 0.0).sum()

# Contar valores diferentes de NaN en la columna
condicion_2 = (~meteo['severerisk'].isna()).sum()

# Contar valores iguales a un valor de cadena concreto en la columna
valor_concreto = 'Partially cloudy'
condicion_3 = (meteo['conditions'] == valor_concreto).sum()

# Imprimir los resultados
print(f'Valores diferentes de 0: {condicion_1}')
print(f'Valores diferentes de NaN: {condicion_2}')
print(f'Valores iguales a "{valor_concreto}": {condicion_3}')"""

In [ ]:
meteo = meteo.loc[~meteo['time'].dt.hour.isin(unique_values)]

In [ ]:
meteo

In [ ]:
#Borramos variables irrelevantes o con 
#meteodrop = meteo.drop(['icon','conditions','name','severerisk','stations','snow', 'snowdepth', 'windgust','dew','preciptype'], axis=1)

meteodrop = meteo[['time','temp','humidity','windspeed']]
meteodrop

#### Dataframe agrupado por horas

In [ ]:
df_agrupado = records.set_index('time')
df_agrupado = df_agrupado.groupby(['stationID', pd.Grouper(freq='H', level=0), 'lineID', 'status'])['userID'].count().unstack(fill_value=0)
df_agrupado.rename(columns={0: 'salida', 1: 'entrada'}, inplace=True)
df_agrupado.reset_index(inplace=True)
mapeo_lineID = {
    'A': 1,
    'B': 2,
    'C': 3,
    # Agrega más mapeos según sea necesario
}

# Aplicar la conversión utilizando el método map
df_agrupado['lineID'] = df_agrupado['lineID'].map(mapeo_lineID)
df_agrupado['lineID'] = df_agrupado['lineID'].astype('category')
df_agrupado

In [ ]:
resultado = df_agrupado.merge(meteodrop, on=['time'], how='left')

In [ ]:
resultado[(resultado['stationID'] == 1) & (resultado['time'].dt.day == 20)]

In [ ]:
resultado[(resultado['stationID']==10) & (resultado['time'].dt.day==20)].corr()

In [ ]:
resultado.corr()

In [ ]:
resultado_fecha_junta_hora = resultado.copy()

In [ ]:
resultado['year'] = resultado['time'].dt.year
resultado['month'] = resultado['time'].dt.month
resultado['day'] = resultado['time'].dt.day
resultado['hour'] = resultado['time'].dt.hour
resultado.drop(columns=['time'], inplace=True)
resultado_hora = resultado.copy()

In [ ]:
resultado_hora

In [ ]:
resultado_fecha_junta_hora

In [ ]:
test_corr = resultado_fecha_junta_hora[(resultado_fecha_junta_hora['stationID']==10) & (resultado_fecha_junta_hora['time'].dt.day==20)]
test_corr.corr().drop(columns=['stationID', 'time', 'lineID']).drop(index=['stationID', 'time', 'lineID'])

#### Dataframe agrupado por intervalos de 10 min

In [ ]:
df_agrupado = records.set_index('time')
#Para agrupar por horas poner 'H' en la frecuencia del Grouper
df_agrupado = df_agrupado.groupby(['stationID', pd.Grouper(freq='10T', level=0),'lineID' ,'status'])['userID'].count().unstack(fill_value=0)
df_agrupado.rename(columns={0: 'salida', 1: 'entrada'}, inplace=True)
df_agrupado.reset_index(inplace=True)
mapeo_lineID = {
    'A': 1,
    'B': 2,
    'C': 3,
    # Agrega más mapeos según sea necesario
}

# Aplicar la conversión utilizando el método map
df_agrupado['lineID'] = df_agrupado['lineID'].map(mapeo_lineID)
df_agrupado['lineID'] = df_agrupado['lineID'].astype('category')
df_agrupado

In [ ]:
resultado = df_agrupado.merge(meteodrop, on=['time'], how='left')

In [ ]:
resultado['temp'].interpolate(method='linear', inplace=True)
resultado['humidity'].interpolate(method='linear', inplace=True)
resultado['windspeed'].interpolate(method='linear', inplace=True)

In [ ]:
resultado_fecha_junta = resultado.copy()

In [ ]:
resultado['year'] = resultado['time'].dt.year
resultado['month'] = resultado['time'].dt.month
resultado['day'] = resultado['time'].dt.day
resultado['hour'] = resultado['time'].dt.hour
resultado['min'] = resultado['time'].dt.minute
resultado.drop(columns=['time'], inplace=True)
resultado

In [ ]:
resultado_fecha_junta

In [ ]:
test_corr = resultado_fecha_junta[(resultado_fecha_junta['stationID']==10) & (resultado_fecha_junta['time'].dt.day==20)]
test_corr.corr().drop(columns=['stationID', 'time', 'lineID']).drop(index=['stationID', 'time', 'lineID'])

In [ ]:
import gc
del df_agrupado
gc.collect()

In [ ]:
del meteo, meteodrop
gc.collect()

In [ ]:
resultado_fecha_junta

In [ ]:
estacion_mas_concurrente = resultado_fecha_junta.groupby('stationID').agg({'salida': 'sum', 'entrada': 'sum'}).reset_index()
# Paso 2: Crea una nueva columna con la suma de 'salida' y 'entrada
estacion_mas_concurrente['total'] = estacion_mas_concurrente['salida'] + estacion_mas_concurrente['entrada']
# Paso 3: Encuentra la estación con el total más grande
estacion_max_total = estacion_mas_concurrente[estacion_mas_concurrente['total'] == estacion_mas_concurrente['total'].max()]
print("La estación con el total más grande para entrada y salida es:")
print(estacion_max_total)

In [ ]:
test = resultado_fecha_junta[resultado_fecha_junta['stationID']==15]

In [ ]:
plt.figure(figsize=(12, 6))  # Establece el tamaño de la gráfica

# Grafica la entrada y la salida en el eje Y contra 'time' en el eje X
plt.plot(test['time'], test['entrada'], label='Entrada', marker='o')
plt.plot(test['time'], test['salida'], label='Salida', marker='o')

# Personaliza la gráfica
plt.xlabel('Tiempo')
plt.ylabel('Valores de Entrada y Salida')
plt.title('Valores de Entrada y Salida a lo largo del tiempo')
plt.legend()

# Rota las etiquetas del eje X para que sean legibles
plt.xticks(rotation=45)

# Muestra la gráfica
plt.show()

In [ ]:
test = resultado_fecha_junta[resultado_fecha_junta['stationID']==80]

In [ ]:
plt.figure(figsize=(12, 6))  # Establece el tamaño de la gráfica

# Grafica la entrada y la salida en el eje Y contra 'time' en el eje X
plt.plot(test['time'], test['entrada'], label='Entrada', marker='o')
plt.plot(test['time'], test['salida'], label='Salida', marker='o')

# Personaliza la gráfica
plt.xlabel('Tiempo')
plt.ylabel('Valores de Entrada y Salida')
plt.title('Valores de Entrada y Salida a lo largo del tiempo')
plt.legend()

# Rota las etiquetas del eje X para que sean legibles
plt.xticks(rotation=45)

# Muestra la gráfica
plt.show()

## Predicciones

In [ ]:
train = records[records['time'].dt.day < 12].copy()
test = records[(records['time'].dt.day < 20) & (records['time'].dt.day > 12)].copy()
pred = records[records['time'].dt.day > 20].copy()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
records

In [ ]:
df_agrupado = records.set_index('time')
df_agrupado = df_agrupado.groupby(['stationID', pd.Grouper(freq='H', level=0), 'status'])['userID'].count().unstack(fill_value=0)
df_agrupado.rename(columns={0: 'salida', 1: 'entrada'}, inplace=True)
df_agrupado

In [ ]:
df_agrupado = df_agrupado.reset_index()

In [ ]:
df_agrupado

In [ ]:
df_agrupado['year'] = df_agrupado['time'].dt.year
df_agrupado['month'] = df_agrupado['time'].dt.month
df_agrupado['day'] = df_agrupado['time'].dt.day
df_agrupado['hour'] = df_agrupado['time'].dt.hour
df_agrupado.drop(columns=['time'], inplace=True)

In [ ]:
df_agrupado.dtypes

In [ ]:
df_agrupado

In [ ]:
X = df_agrupado[['stationID', 'year', 'month', 'day', 'hour']]
y_entrada = df_agrupado['entrada']
y_salida = df_agrupado['salida']

## REGRESIÓN LINEAL

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# Divide tus datos en un conjunto de entrenamiento y un conjunto de prueba
X_train, X_test, y_entrada_train, y_entrada_test, y_salida_train, y_salida_test = train_test_split(X, y_entrada, y_salida, test_size=0.2, random_state=42)

# Crea un modelo de regresión lineal para las entradas
modelo_entrada = LinearRegression()
modelo_entrada.fit(X_train, y_entrada_train)

# Realiza predicciones para las entradas
predicciones_entrada = modelo_entrada.predict(X_test)
predicciones_entrada = np.round(predicciones_entrada).astype(int)

# Calcula el error para las predicciones de entradas
error_entrada = mean_squared_error(y_entrada_test, predicciones_entrada)
print(f"Error de entradas: {error_entrada}")

# Crea un modelo de regresión lineal para las salidas
modelo_salida = LinearRegression()
modelo_salida.fit(X_train, y_salida_train)

# Realiza predicciones para las salidas
predicciones_salida = modelo_salida.predict(X_test)
predicciones_salida = np.round(predicciones_salida).astype(int)

# Calcula el error para las predicciones de salidas
error_salida = np.sqrt(mean_squared_error(y_salida_test, predicciones_salida))
print(f"Error de salidas: {error_salida}")

In [ ]:
# Divide tus datos en un conjunto de entrenamiento y un conjunto de prueba
X_train, X_test, y_entrada_train, y_entrada_test, y_salida_train, y_salida_test = train_test_split(X, y_entrada, y_salida, test_size=0.2, random_state=42)

# Crea un modelo de regresión lineal para las entradas
modelo_entrada = LinearRegression()
modelo_entrada.fit(X_train, y_entrada_train)

# Realiza predicciones para las entradas
predicciones_entrada = modelo_entrada.predict(X_test)

# Calcula el error para las predicciones de entradas
error_entrada = mean_squared_error(y_entrada_test, predicciones_entrada)
print(f"Error de entradas: {error_entrada}")

# Crea un modelo de regresión lineal para las salidas
modelo_salida = LinearRegression()
modelo_salida.fit(X_train, y_salida_train)

# Realiza predicciones para las salidas
predicciones_salida = modelo_salida.predict(X_test)

# Calcula el error para las predicciones de salidas
error_salida = np.sqrt(mean_squared_error(y_salida_test, predicciones_salida))
print(f"Error de salidas: {error_salida}")

In [ ]:
predicciones_salida.size

### Cada 10 min y Meteo incluido

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler

In [ ]:
resultado

In [ ]:
scaler = MinMaxScaler()
X = resultado[['stationID', 'lineID', 'temp', 'humidity', 'precip', 'windspeed']]
y_entrada = resultado['entrada']
y_salida = resultado['salida']
X_scaled = scaler.fit_transform(X)

In [ ]:
X_scaled

In [ ]:
# Divide tus datos en un conjunto de entrenamiento y un conjunto de prueba
X_train, X_test, y_entrada_train, y_entrada_test, y_salida_train, y_salida_test = train_test_split(X_scaled, y_entrada, y_salida, test_size=0.2, random_state=42)

# Crea un modelo de regresión lineal para las entradas
modelo_entrada = LinearRegression()
modelo_entrada.fit(X_train, y_entrada_train)

# Realiza predicciones para las entradas
predicciones_entrada = modelo_entrada.predict(X_test)
predicciones_entrada = np.round(predicciones_entrada).astype(int)

# Calcula el error para las predicciones de entradas
error_entrada =  np.sqrt(mean_squared_error(y_entrada_test, predicciones_entrada))
print(f"Error de entradas: {error_entrada}")

# Crea un modelo de regresión lineal para las salidas
modelo_salida = LinearRegression()
modelo_salida.fit(X_train, y_salida_train)

# Realiza predicciones para las salidas
predicciones_salida = modelo_salida.predict(X_test)
predicciones_salida = np.round(predicciones_salida).astype(int)

# Calcula el error para las predicciones de salidas
error_salida = np.sqrt(mean_squared_error(y_salida_test, predicciones_salida))
print(f"Error de salidas: {error_salida}")

## LONG-SHORT TERM MEMORY (LSTM)

### 1 Hora 1 estación sin METEO

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import random

In [ ]:
def reset_random_seeds():
   os.environ['PYTHONHASHSEED']=str(2)
   tf.random.set_seed(2)
   np.random.seed(2)
   random.seed(2)

In [ ]:
resultado_fecha_junta_hora

In [ ]:
station_data = resultado_fecha_junta_hora[resultado_fecha_junta_hora['stationID'] == 0]
station_data = station_data.drop(columns=['stationID','temp','humidity','windspeed','lineID'])
station_data

In [ ]:
# Escalar los datos
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(station_data[['entrada', 'salida']])

# Definir la longitud de la secuencia temporal
seq_length = 18  # Por ejemplo, utilizar las últimas 24 horas para predecir las próximas horas, pero utilizamos 18 porque no se toman datos de las 00:00 a las 04:00

# Crear secuencias temporales y etiquetas
def create_sequences(data, seq_length):
    sequences = []
    labels = []
    for i in range(len(data) - seq_length):
        seq = data[i:i+seq_length]
        label = data[i+seq_length]        
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

X, y = create_sequences(scaled_data, seq_length)

In [ ]:
# Dividir los datos en entrenamiento y prueba
train_size = int(len(X) * 0.8)  # Puedes ajustar la proporción según tus necesidades
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Crear y entrenar el modelo LSTM
model = Sequential()
model.random_state = 42  # Reemplaza 42 con la semilla que desees
model.add(LSTM(300, activation='relu', input_shape=(seq_length, 2)))  # 2 características: 'entrada' y 'salida'
model.add(Dense(2))  # 2 salidas para 'entrada' y 'salida'

model.compile(optimizer='adam', loss='mse')  # Puedes usar otra función de pérdida según tu problema

model.fit(X_train, y_train, epochs=300, batch_size=seq_length)  # Ajusta los hiperparámetros según tus necesidades

# Realizar predicciones
predicted_data = model.predict(X_test)

In [ ]:
# Invertir la escala de las predicciones para obtener valores reales
predicted_data = scaler.inverse_transform(predicted_data)
# Desescalar los datos de prueba y las predicciones para obtener valores reales
y_test_actual = scaler.inverse_transform(y_test)
# Crear el gráfico
time_range = range(len(y_test_actual))
plt.figure(figsize=(12, 6))
plt.plot(time_range, y_test_actual[:, 0], label='Valor Real de Entrada', marker='o', linestyle='-')
plt.plot(time_range, predicted_data[:, 0], label='Predicción de Entrada', marker='o', linestyle='--')
plt.plot(time_range, y_test_actual[:, 1], label='Valor Real de Salida', marker='o', linestyle='-')
plt.plot(time_range, predicted_data[:, 1], label='Predicción de Salida', marker='o', linestyle='--')

plt.xlabel('Tiempo')
plt.ylabel('Valor')
plt.title('Predicción de Entrada y Salida para la estación 0')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(time_range, y_test_actual[:, 0], label='Valor Real de Entrada', marker='o', linestyle='-')
plt.plot(time_range, predicted_data[:, 0], label='Predicción de Entrada', marker='o', linestyle='--')

plt.xlabel('Tiempo')
plt.ylabel('Valor')
plt.title('Predicción de Entrada para la Estación 0')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(time_range, y_test_actual[:, 1], label='Valor Real de Salida', marker='o', linestyle='-')
plt.plot(time_range, predicted_data[:, 1], label='Predicción de Salida', marker='o', linestyle='--')

plt.xlabel('Tiempo')
plt.ylabel('Valor')
plt.title('Predicción de Salida para la Estación 0')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Calcula el RMSE
rmse = np.sqrt(mean_squared_error(y_test_actual, predicted_data))

# Calcula el coeficiente de determinación (R-cuadrado)
r2 = r2_score(y_test_actual, predicted_data)

print("RMSE:", rmse)
print("R-cuadrado:", r2)

### 1 Hora 1 estación con METEO

In [ ]:
resultado_fecha_junta_hora

In [ ]:
station_data = resultado_fecha_junta_hora[resultado_fecha_junta_hora['stationID'] == 0]

# Escalar los datos
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(station_data[['entrada', 'salida', 'temp', 'humidity', 'windspeed']])

# Escalar los datos a predecir
scaler_y = MinMaxScaler()
scaler_y.fit_transform(station_data[['entrada', 'salida']])

# Definir la longitud de la secuencia temporal
seq_length = 18  # Por ejemplo, utilizar las últimas 24 horas para predecir las próximas horas, pero utilizamos 18 porque no se toman datos de las 00:00 a las 04:00

# Crear secuencias temporales y etiquetas
def create_sequences(data, seq_length):
    sequences = []
    labels = []
    for i in range(len(data) - seq_length):
        seq = data[i:i+seq_length]
        label = data[i+seq_length]        
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

X, y = create_sequences(scaled_data, seq_length)

y = y[:, :-3] #Solo queremos predecir las dos primeras variables, entrada y salida

In [ ]:
# Dividir los datos en entrenamiento y prueba
train_size = int(len(X) * 0.8)  # Puedes ajustar la proporción según tus necesidades
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

reset_random_seeds()
# Crear y entrenar el modelo LSTM
model = Sequential()
model.add(LSTM(500, activation='relu', input_shape=(seq_length, 5)))  # 2 características: 'entrada' y 'salida'
model.add(Dense(2))  # 2 salidas para 'entrada' y 'salida'
model.compile(optimizer='adam', loss='mse')  # Puedes usar otra función de pérdida según tu problema
model.fit(X_train, y_train, epochs=200, batch_size=seq_length)  # Ajusta los hiperparámetros según tus necesidades

# Realizar predicciones
predicted_data = model.predict(X_test)

In [ ]:
# Invertir la escala de las predicciones para obtener valores reales
predicted_data = scaler_y.inverse_transform(predicted_data)
# Desescalar los datos de prueba y las predicciones para obtener valores reales
y_test_actual = scaler_y.inverse_transform(y_test)
# Crear el gráfico
time_range = range(len(y_test_actual))
plt.figure(figsize=(12, 6))
plt.plot(time_range, y_test_actual[:, 0], label='Valor Real de Entrada', marker='o', linestyle='-')
plt.plot(time_range, predicted_data[:, 0], label='Predicción de Entrada', marker='o', linestyle='--')
plt.plot(time_range, y_test_actual[:, 1], label='Valor Real de Salida', marker='o', linestyle='-')
plt.plot(time_range, predicted_data[:, 1], label='Predicción de Salida', marker='o', linestyle='--')

plt.xlabel('Tiempo')
plt.ylabel('Valor')
plt.title('Predicción de Entrada y Salida para la estación 0')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Calcula el RMSE
rmse = np.sqrt(mean_squared_error(y_test_actual, predicted_data))

# Calcula el coeficiente de determinación (R-cuadrado)
r2 = r2_score(y_test_actual, predicted_data)

print("RMSE:", rmse)
print("R-cuadrado:", r2)

### 10 Min 1 estación Sin METEO

In [ ]:
resultado_fecha_junta

In [ ]:
station_data = resultado_fecha_junta[resultado_fecha_junta['stationID'] == 0]
station_data = station_data.drop(columns=['stationID','temp','humidity','windspeed','lineID'])
station_data

In [ ]:
# Escalar los datos
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(station_data[['entrada', 'salida']])

# Definir la longitud de la secuencia temporal
seq_length = 108  # Por ejemplo, utilizar las últimas 24 horas para predecir las próximas horas, pero utilizamos 18 porque no se toman datos de las 00:00 a las 04:00

# Crear secuencias temporales y etiquetas
def create_sequences(data, seq_length):
    sequences = []
    labels = []
    for i in range(len(data) - seq_length):
        seq = data[i:i+seq_length]
        label = data[i+seq_length]        
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

X, y = create_sequences(scaled_data, seq_length)

In [ ]:
# Dividir los datos en entrenamiento y prueba
train_size = int(len(X) * 0.8)  # Puedes ajustar la proporción según tus necesidades
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Crear y entrenar el modelo LSTM
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(seq_length, 2)))  # 2 características: 'entrada' y 'salida'
model.add(Dense(2))  # 2 salidas para 'entrada' y 'salida'

model.compile(optimizer='adam', loss='mse')  # Puedes usar otra función de pérdida según tu problema

model.fit(X_train, y_train, epochs=50, batch_size=seq_length)  # Ajusta los hiperparámetros según tus necesidades

# Realizar predicciones
predicted_data = model.predict(X_test)

In [ ]:
# Invertir la escala de las predicciones para obtener valores reales
predicted_data = scaler.inverse_transform(predicted_data)
# Desescalar los datos de prueba y las predicciones para obtener valores reales
y_test_actual = scaler.inverse_transform(y_test)
# Crear el gráfico
time_range = range(len(y_test_actual))
plt.figure(figsize=(12, 6))
plt.plot(time_range, y_test_actual[:, 0], label='Valor Real de Entrada', marker='o', linestyle='-')
plt.plot(time_range, predicted_data[:, 0], label='Predicción de Entrada', marker='o', linestyle='--')
plt.plot(time_range, y_test_actual[:, 1], label='Valor Real de Salida', marker='o', linestyle='-')
plt.plot(time_range, predicted_data[:, 1], label='Predicción de Salida', marker='o', linestyle='--')

plt.xlabel('Tiempo')
plt.ylabel('Valor')
plt.title('Predicción de Entrada y Salida para la estación 0')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Calcula el RMSE
rmse = np.sqrt(mean_squared_error(y_test_actual, predicted_data))

# Calcula el coeficiente de determinación (R-cuadrado)
r2 = r2_score(y_test_actual, predicted_data)

print("RMSE:", rmse)
print("R-cuadrado:", r2)

### 10 Min 1 estación con METEO

In [ ]:
resultado_fecha_junta

In [ ]:
station_data = resultado_fecha_junta[resultado_fecha_junta['stationID'] == 0]

# Escalar los datos
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(station_data[['entrada', 'salida', 'temp', 'humidity', 'windspeed']])

# Escalar los datos a predecir
scaler_y = MinMaxScaler()
scaler_y.fit_transform(station_data[['entrada', 'salida']])

# Definir la longitud de la secuencia temporal
seq_length = 108  # Por ejemplo, utilizar las últimas 24 horas para predecir las próximas horas, pero utilizamos 18 porque no se toman datos de las 00:00 a las 04:00

# Crear secuencias temporales y etiquetas
def create_sequences(data, seq_length):
    sequences = []
    labels = []
    for i in range(len(data) - seq_length):
        seq = data[i:i+seq_length]
        label = data[i+seq_length]        
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

X, y = create_sequences(scaled_data, seq_length)

y = y[:, :-3] #Solo queremos predecir las dos primeras variables, entrada y salida

# Dividir los datos en entrenamiento y prueba
train_size = int(len(X) * 0.8)  # Puedes ajustar la proporción según tus necesidades
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Crear y entrenar el modelo LSTM
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(seq_length, 5)))  #5 características
model.add(Dense(2))  # 2 salidas para 'entrada' y 'salida'

model.compile(optimizer='adam', loss='mse')  # Puedes usar otra función de pérdida según tu problema

model.fit(X_train, y_train, epochs=50, batch_size=seq_length)  # Ajusta los hiperparámetros según tus necesidades

# Realizar predicciones
predicted_data = model.predict(X_test)

# Invertir la escala de las predicciones para obtener valores reales
predicted_data = scaler_y.inverse_transform(predicted_data)
# Desescalar los datos de prueba y las predicciones para obtener valores reales
y_test_actual = scaler_y.inverse_transform(y_test)
# Crear el gráfico
time_range = range(len(y_test_actual))
plt.figure(figsize=(12, 6))
plt.plot(time_range, y_test_actual[:, 0], label='Valor Real de Entrada', marker='o', linestyle='-')
plt.plot(time_range, predicted_data[:, 0], label='Predicción de Entrada', marker='o', linestyle='--')
plt.plot(time_range, y_test_actual[:, 1], label='Valor Real de Salida', marker='o', linestyle='-')
plt.plot(time_range, predicted_data[:, 1], label='Predicción de Salida', marker='o', linestyle='--')

plt.xlabel('Tiempo')
plt.ylabel('Valor')
plt.title('Predicción de Entrada y Salida para la estación 0')
plt.legend()
plt.grid(True)
plt.show()

# Calcula el RMSE
rmse = np.sqrt(mean_squared_error(y_test_actual, predicted_data))

# Calcula el coeficiente de determinación (R-cuadrado)
r2 = r2_score(y_test_actual, predicted_data)

print("RMSE:", rmse)
print("R-cuadrado:", r2)

In [ ]:
# Calcula el RMSE
rmse = np.sqrt(mean_squared_error(y_test_actual, predicted_data))

# Calcula el coeficiente de determinación (R-cuadrado)
r2 = r2_score(y_test_actual, predicted_data)

#Calcula el MAPE
# Encuentra el valor mínimo y máximo de los datos reales y de predicción
min_real = np.min(y_test_actual)
max_real = np.max(y_test_actual)
min_pred = np.min(predicted_data)
max_pred = np.max(predicted_data)

# Escala los datos reales y de predicción al rango de 1 a 10
scaled_real = 1 + (9 * (y_test_actual - min_real) / (max_real - min_real))
scaled_pred = 1 + (9 * (predicted_data - min_pred) / (max_pred - min_pred))

n = len(scaled_real)
mape = (1/n) * np.sum(np.abs((scaled_real - scaled_pred) / scaled_real) * 100)

print("RMSE:", rmse)
print("R-cuadrado:", r2)
print("MAPE:", mape)

### Prueba con todas las estaciones

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
estaciones = resultado_fecha_junta['stationID'].unique()

In [ ]:
best_rmse = None
best_r2 = None
best_station = None
mape_arr = []
for est in estaciones:

    station_data = resultado_fecha_junta[resultado_fecha_junta['stationID'] == est]

    # Escalar los datos
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(station_data[['entrada', 'salida', 'temp', 'humidity', 'windspeed']])

    # Escalar los datos a predecir
    scaler_y = MinMaxScaler()
    scaler_y.fit_transform(station_data[['entrada', 'salida']])

    # Definir la longitud de la secuencia temporal
    seq_length = 108  # Por ejemplo, utilizar las últimas 24 horas para predecir las próximas horas, pero utilizamos 18 porque no se toman datos de las 00:00 a las 04:00

    # Crear secuencias temporales y etiquetas
    def create_sequences(data, seq_length):
        sequences = []
        labels = []
        for i in range(len(data) - seq_length):
            seq = data[i:i+seq_length]
            label = data[i+seq_length]        
            sequences.append(seq)
            labels.append(label)
        return np.array(sequences), np.array(labels)

    X, y = create_sequences(scaled_data, seq_length)

    y = y[:, :-4] #Solo queremos predecir las dos primeras variables, entrada y salida

    # Dividir los datos en entrenamiento y prueba
    train_size = int(len(X) * 0.8)  # Puedes ajustar la proporción según tus necesidades
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    # Crear y entrenar el modelo LSTM
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(seq_length, 5)))  #6 características
    model.add(Dense(2))  # 2 salidas para 'entrada' y 'salida'

    model.compile(optimizer='adam', loss='mse')  # Puedes usar otra función de pérdida según tu problema

    model.fit(X_train, y_train, epochs=50, batch_size=seq_length)  # Ajusta los hiperparámetros según tus necesidades

    # Realizar predicciones
    predicted_data = model.predict(X_test)

    # Invertir la escala de las predicciones para obtener valores reales
    predicted_data = scaler_y.inverse_transform(predicted_data)
    # Desescalar los datos de prueba y las predicciones para obtener valores reales
    y_test_actual = scaler_y.inverse_transform(y_test)

    # Calcula el RMSE
    rmse = np.sqrt(mean_squared_error(y_test_actual, predicted_data))

    # Calcula el coeficiente de determinación (R-cuadrado)
    r2 = r2_score(y_test_actual, predicted_data)

    #Calcula el MAPE
    # Encuentra el valor mínimo y máximo de los datos reales y de predicción
    min_real = np.min(y_test_actual)
    max_real = np.max(y_test_actual)
    min_pred = np.min(predicted_data)
    max_pred = np.max(predicted_data)

    # Escala los datos reales y de predicción al rango de 1 a 10
    scaled_real = 1 + (9 * (y_test_actual - min_real) / (max_real - min_real))
    scaled_pred = 1 + (9 * (predicted_data - min_pred) / (max_pred - min_pred))

    n = len(scaled_real)
    mape = (1/n) * np.sum(np.abs((scaled_real - scaled_pred) / scaled_real) * 100)

    mape_arr.append(mape)

    if(best_rmse is None or rmse < best_rmse):
        best_rmse = rmse
        best_r2 = r2
        best_station = est

mape_total = np.mean(mape_arr)
mape_total

## ARIMA

In [ ]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from pmdarima.arima import auto_arima

### Intento manual

In [ ]:
arima_df = records.set_index('time')
arima_df = arima_df.groupby(['stationID', pd.Grouper(freq='H', level=0), 'status'])['userID'].count().unstack(fill_value=0)
arima_df.rename(columns={0: 'salida', 1: 'entrada'}, inplace=True)
arima_df.reset_index(inplace=True)
arima_df.set_index('time', inplace=True)

In [ ]:
arima_df

In [ ]:
station_data = arima_df[arima_df['stationID'] == 0]
station_data

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(station_data["salida"], label="Salida")
plt.plot(station_data["entrada"], label="Entrada")
plt.xlabel("Fecha y Hora")
plt.ylabel("Cantidad")
plt.legend()
plt.title("Datos de Salida y Entrada por Hora")
plt.show()

In [ ]:
# Realiza una prueba de estacionariedad
def adf_test(series):
    result = adfuller(series, autolag="AIC")
    print("ADF Statistic:", result[0])
    print("p-value:", result[1])
    print("Critical Values:", result[4])

adf_test(station_data["salida"])  # Realiza la prueba para la serie de salida
adf_test(station_data["entrada"])  # Realiza la prueba para la serie de entrada


In [ ]:
# Diferencia los datos si es necesario para hacerlos estacionarios
station_data['salida_diff'] = station_data['salida'].diff().dropna()
station_data['entrada_diff'] = station_data['entrada'].diff().dropna()

In [ ]:
# Visualiza las series diferenciadas
plt.figure(figsize=(12, 6))
plt.plot(station_data["salida_diff"], label="Diferencia de Salida")
plt.plot(station_data["entrada_diff"], label="Diferencia de Entrada")
plt.xlabel("Fecha y Hora")
plt.ylabel("Diferencia de Cantidad")
plt.legend()
plt.title("Series Diferenciadas de Salida y Entrada por Hora")
plt.show()

In [ ]:
# Realiza un análisis de autocorrelación y autocorrelación parcial
plot_acf(station_data["salida_diff"], lags=40)
plot_pacf(station_data["salida_diff"], lags=40)
plt.show()

In [ ]:
plot_acf(station_data["entrada_diff"], lags=40)
plot_pacf(station_data["entrada_diff"], lags=40)
plt.show()

In [ ]:
# Ajusta el modelo ARIMA
model_salida = ARIMA(station_data["salida"], order=(1, 1, 1))
model_entrada = ARIMA(station_data["entrada"], order=(1, 1, 1))

In [ ]:
# Ajusta el modelo a los datos
model_salida_fit = model_salida.fit()
model_entrada_fit = model_entrada.fit()

In [ ]:
# Realiza predicciones
n_forecast = 24  # Número de pasos hacia adelante a predecir

forecast_salida = model_salida_fit.forecast(steps=n_forecast)
forecast_entrada = model_entrada_fit.forecast(steps=n_forecast)

In [ ]:
# Visualiza las predicciones
plt.figure(figsize=(12, 6))
plt.plot(station_data.index, station_data["salida"], label="Salida (Observado)")
plt.plot(pd.date_range(start=station_data.index[-1], periods=n_forecast, freq="H"), forecast_salida, label="Salida (Predicción)")
plt.xlabel("Fecha y Hora")
plt.ylabel("Cantidad")
plt.legend()
plt.title("Predicción de Salida por Hora")
plt.show()

plt.figure(figsize=(12, 6))
plt.plot(station_data.index, station_data["entrada"], label="Entrada (Observado)")
plt.plot(pd.date_range(start=station_data.index[-1], periods=n_forecast, freq="H"), forecast_entrada, label="Entrada (Predicción)")
plt.xlabel("Fecha y Hora")
plt.ylabel("Cantidad")
plt.legend()
plt.title("Predicción de Entrada por Hora")
plt.show()

### Intento automático

In [ ]:
arima_df = records.set_index('time')
arima_df = arima_df.groupby(['stationID', pd.Grouper(freq='H', level=0), 'status'])['userID'].count().unstack(fill_value=0)
arima_df.rename(columns={0: 'salida', 1: 'entrada'}, inplace=True)
arima_df.reset_index(inplace=True)
arima_df.set_index('time', inplace=True)

In [ ]:
station_data = arima_df[arima_df['stationID'] == 0]
station_data

In [ ]:
# Separa tus datos en conjuntos de entrenamiento y prueba
train_size = int(len(station_data) * 0.8)
train_data = station_data.iloc[:train_size]
test_data = station_data.iloc[train_size:]

In [ ]:
model_entrada = auto_arima(train_data['entrada'], seasonal=True, stepwise=True, trace=True)

In [ ]:
model_salida = auto_arima(train_data['salida'], seasonal=True, stepwise=True, trace=True)

In [ ]:
# Ajusta el modelo ARIMA con los parámetros óptimos encontrados
model_entrada.fit(train_data['entrada'])
model_salida.fit(train_data['salida'])

In [ ]:
# Realiza predicciones en el conjunto de prueba
predictions_entrada = model_entrada.predict(n_periods=len(test_data))
predictions_salida = model_salida.predict(n_periods=len(test_data))

In [ ]:
# Visualiza las predicciones
plt.figure(figsize=(12, 6))
plt.plot(test_data.index, test_data["salida"], label="Salida (Observado)")
plt.plot(pd.date_range(start=test_data.index[-1], periods=len(test_data), freq="H"), predictions_salida, label="Salida (Predicción)")
plt.xlabel("Fecha y Hora")
plt.ylabel("Cantidad")
plt.legend()
plt.title("Predicción de Salida por Hora")
plt.show()

plt.figure(figsize=(12, 6))
plt.plot(test_data.index, test_data["entrada"], label="Entrada (Observado)")
plt.plot(pd.date_range(start=test_data.index[-1], periods=len(test_data), freq="H"), predictions_entrada, label="Entrada (Predicción)")
plt.xlabel("Fecha y Hora")
plt.ylabel("Cantidad")
plt.legend()
plt.title("Predicción de Entrada por Hora")
plt.show()

In [ ]:
rmse_entrada = np.sqrt(mean_squared_error(test_data['entrada'], predictions_entrada))
print(f'RMSE: {rmse_entrada}')
rmse_salida = np.sqrt(mean_squared_error(test_data['salida'], predictions_salida))
print(f'RMSE: {rmse_salida}')

In [ ]:
del arima_df
gc.collect()

## PROPHET

### 1 Hora Estación única sin METEO 

#### Salida

In [ ]:
from prophet import Prophet
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
resultado_fecha_junta_hora

In [ ]:
station_data = resultado_fecha_junta_hora[resultado_fecha_junta_hora['stationID'] == 0]
station_data = station_data.drop(columns=['stationID', 'entrada','temp','humidity','windspeed','lineID'])
station_data.rename(columns={'time': 'ds', 'salida': 'y'},inplace=True)

train_size = int(len(station_data) * 0.8)
train_data_salida = station_data.iloc[:train_size]
test_data_salida = station_data.iloc[train_size:]

# Crear un modelo Prophet
model = Prophet()
model.random_state = 42  # Reemplaza 42 con la semilla que desees

# model.add_regressor('lineID')
# model.add_regressor('temp')
# model.add_regressor('humidity')
# model.add_regressor('precip')
# model.add_regressor('windspeed')

# Ajustar el modelo a los datos
model.fit(train_data_salida)
# Realizar las predicciones
forecast_salida = model.predict(test_data_salida)
# Visualizar las predicciones
rmse_salida = np.sqrt(mean_squared_error(test_data_salida['y'], forecast_salida['yhat']))
print(f'RMSE: {rmse_salida}')
r2_salida = r2_score(test_data_salida['y'],forecast_salida['yhat'])
print(f'R^2: {r2_salida}')

In [ ]:
fig = model.plot(forecast_salida)
plt.show()

#### Entrada

In [ ]:
station_data = resultado_fecha_junta_hora[resultado_fecha_junta_hora['stationID'] == 0]
station_data = station_data.drop(columns=['stationID', 'salida','temp','humidity','windspeed','lineID'])
station_data.rename(columns={'time': 'ds', 'entrada': 'y'},inplace=True)

train_size = int(len(station_data) * 0.8)
train_data_entrada = station_data.iloc[:train_size]
test_data_entrada = station_data.iloc[train_size:]

# Crear un modelo Prophet
model = Prophet()
model.random_state = 42  # Reemplaza 42 con la semilla que desees
# model.add_regressor('lineID')
# model.add_regressor('temp')
# model.add_regressor('humidity')
# model.add_regressor('precip')
# model.add_regressor('windspeed')

# Ajustar el modelo a los datos
model.fit(train_data_entrada)
# Realizar las predicciones
forecast_entrada = model.predict(test_data_entrada)
rmse_entrada = np.sqrt(mean_squared_error(test_data_entrada['y'], forecast_entrada['yhat']))
print(f'RMSE: {rmse_entrada}')
r2_entrada = r2_score(test_data_entrada['y'],forecast_entrada['yhat'])
print(f'R^2: {r2_entrada}')

In [ ]:
comb_pred = np.array([forecast_entrada['yhat'], forecast_salida['yhat']]).T
comb_real = np.array([test_data_entrada['y'], test_data_salida['y']]).T
rmse = np.sqrt(mean_squared_error(comb_real, comb_pred))
r2 = r2_score(comb_real, comb_pred)
print("RMSE combinado:", rmse)
print("R² combinado:", r2)

### 1 hora Estación única con METEO

#### Salida

In [ ]:
station_data = resultado_fecha_junta_hora[resultado_fecha_junta_hora['stationID'] == 0]
#station_data = station_data.drop(columns=['stationID', 'entrada','temp','humidity','precip','windspeed','lineID'])
station_data.rename(columns={'time': 'ds', 'salida': 'y'},inplace=True)

train_size = int(len(station_data) * 0.8)
train_data_salida = station_data.iloc[:train_size]
test_data_salida = station_data.iloc[train_size:]
# Crear un modelo Prophet
model = Prophet()
model.random_state = 42  # Reemplaza 42 con la semilla que desees
model.add_regressor('lineID')
model.add_regressor('entrada')
model.add_regressor('temp')
model.add_regressor('humidity')
model.add_regressor('windspeed')

# Ajustar el modelo a los datos
model.fit(train_data_salida)
# Realizar las predicciones
forecast_salida = model.predict(test_data_salida)
rmse_salida = np.sqrt(mean_squared_error(test_data_salida['y'], forecast_salida['yhat']))
print(f'RMSE: {rmse_salida}')
r2_salida = r2_score(test_data_salida['y'],forecast_salida['yhat'])
print(f'R^2: {r2_salida}')

#### Entrada

In [ ]:
station_data = resultado_fecha_junta_hora[resultado_fecha_junta_hora['stationID'] == 0]
#station_data = station_data.drop(columns=['stationID', 'entrada','temp','humidity','precip','windspeed','lineID'])
station_data.rename(columns={'time': 'ds', 'entrada': 'y'},inplace=True)

train_size = int(len(station_data) * 0.8)
train_data_entrada = station_data.iloc[:train_size]
test_data_entrada = station_data.iloc[train_size:]
# Crear un modelo Prophet
model = Prophet()
model.random_state = 42  # Reemplaza 42 con la semilla que desees
model.add_regressor('lineID')
model.add_regressor('salida')
model.add_regressor('temp')
model.add_regressor('humidity')
model.add_regressor('windspeed')

# Ajustar el modelo a los datos
model.fit(train_data_entrada)
# Realizar las predicciones
forecast_entrada = model.predict(test_data_entrada)
rmse_entrada = np.sqrt(mean_squared_error(test_data_entrada['y'], forecast_entrada['yhat']))
print(f'RMSE: {rmse_entrada}')
r2_entrada = r2_score(test_data_entrada['y'],forecast_entrada['yhat'])
print(f'R^2: {r2_entrada}')

In [ ]:
comb_pred = np.array([forecast_entrada['yhat'], forecast_salida['yhat']]).T
comb_real = np.array([test_data_entrada['y'], test_data_salida['y']]).T
rmse = np.sqrt(mean_squared_error(comb_real, comb_pred))
r2 = r2_score(comb_real, comb_pred)
print("RMSE combinado:", rmse)
print("R² combinado:", r2)

### 10 Min Estación única sin METEO

In [ ]:
resultado_fecha_junta

#### Salida

In [ ]:
station_data = resultado_fecha_junta[resultado_fecha_junta['stationID'] == 0]
station_data = station_data.drop(columns=['stationID', 'entrada','temp','humidity','windspeed','lineID'])
station_data.rename(columns={'time': 'ds', 'salida': 'y'},inplace=True)

train_size = int(len(station_data) * 0.8)
train_data_salida = station_data.iloc[:train_size]
test_data_salida = station_data.iloc[train_size:]

# Crear un modelo Prophet
model = Prophet()
model.random_state = 42  # Reemplaza 42 con la semilla que desees
# model.add_regressor('lineID')
# model.add_regressor('temp')
# model.add_regressor('humidity')
# model.add_regressor('precip')
# model.add_regressor('windspeed')

# Ajustar el modelo a los datos
model.fit(train_data_salida)
# Realizar las predicciones
forecast_salida = model.predict(test_data_salida)
# Visualizar las predicciones
rmse_salida = np.sqrt(mean_squared_error(test_data_salida['y'], forecast_salida['yhat']))
print(f'RMSE: {rmse_salida}')
r2_salida = r2_score(test_data_salida['y'],forecast_salida['yhat'])
print(f'R^2: {r2_salida}')

#### Entrada

In [ ]:
station_data = resultado_fecha_junta[resultado_fecha_junta['stationID'] == 0]
station_data = station_data.drop(columns=['stationID', 'salida','temp','humidity','windspeed','lineID'])
station_data.rename(columns={'time': 'ds', 'entrada': 'y'},inplace=True)

train_size = int(len(station_data) * 0.8)
train_data_entrada = station_data.iloc[:train_size]
test_data_entrada = station_data.iloc[train_size:]

# Crear un modelo Prophet
model = Prophet()
model.random_state = 42  # Reemplaza 42 con la semilla que desees
# model.add_regressor('lineID')
# model.add_regressor('temp')
# model.add_regressor('humidity')
# model.add_regressor('precip')
# model.add_regressor('windspeed')

# Ajustar el modelo a los datos
model.fit(train_data_entrada)
# Realizar las predicciones
forecast_entrada = model.predict(test_data_entrada)
rmse_entrada = np.sqrt(mean_squared_error(test_data_entrada['y'], forecast_entrada['yhat']))
print(f'RMSE: {rmse_entrada}')
r2_entrada = r2_score(test_data_entrada['y'],forecast_entrada['yhat'])
print(f'R^2: {r2_entrada}')

In [ ]:
comb_pred = np.array([forecast_entrada['yhat'], forecast_salida['yhat']]).T
comb_real = np.array([test_data_entrada['y'], test_data_salida['y']]).T
rmse = np.sqrt(mean_squared_error(comb_real, comb_pred))
r2 = r2_score(comb_real, comb_pred)
print("RMSE combinado:", rmse)
print("R² combinado:", r2)

### 10 Min Estación única con METEO

#### Salida

In [ ]:
station_data = resultado_fecha_junta[resultado_fecha_junta['stationID'] == 0]
#station_data = station_data.drop(columns=['stationID', 'entrada','temp','humidity','precip','windspeed','lineID'])
station_data.rename(columns={'time': 'ds', 'salida': 'y'},inplace=True)

train_size = int(len(station_data) * 0.8)
train_data_salida = station_data.iloc[:train_size]
test_data_salida = station_data.iloc[train_size:]
# Crear un modelo Prophet
model = Prophet()
model.random_state = 42  # Reemplaza 42 con la semilla que desees
model.add_regressor('lineID')
model.add_regressor('entrada')
model.add_regressor('temp')
model.add_regressor('humidity')
model.add_regressor('windspeed')

# Ajustar el modelo a los datos
model.fit(train_data_salida)
# Realizar las predicciones
forecast_salida = model.predict(test_data_salida)
rmse_salida = np.sqrt(mean_squared_error(test_data_salida['y'], forecast_salida['yhat']))
print(f'RMSE: {rmse_salida}')
r2_salida = r2_score(test_data_salida['y'],forecast_salida['yhat'])
print(f'R^2: {r2_salida}')

#### Entrada

In [ ]:
station_data = resultado_fecha_junta[resultado_fecha_junta['stationID'] == 0]
#station_data = station_data.drop(columns=['stationID', 'entrada','temp','humidity','precip','windspeed','lineID'])
station_data.rename(columns={'time': 'ds', 'entrada': 'y'},inplace=True)

train_size = int(len(station_data) * 0.8)
train_data_entrada = station_data.iloc[:train_size]
test_data_entrada = station_data.iloc[train_size:]
# Crear un modelo Prophet
model = Prophet()
model.random_state = 42  # Reemplaza 42 con la semilla que desees
model.add_regressor('lineID')
model.add_regressor('salida')
model.add_regressor('temp')
model.add_regressor('humidity')
model.add_regressor('windspeed')

# Ajustar el modelo a los datos
model.fit(train_data_entrada)
# Realizar las predicciones
forecast_entrada = model.predict(test_data_entrada)
rmse_entrada = np.sqrt(mean_squared_error(test_data_entrada['y'], forecast_entrada['yhat']))
print(f'RMSE: {rmse_entrada}')
r2_entrada = r2_score(test_data_entrada['y'],forecast_entrada['yhat'])
print(f'R^2: {r2_entrada}')

In [ ]:
comb_pred = np.array([forecast_entrada['yhat'], forecast_salida['yhat']]).T
comb_real = np.array([test_data_entrada['y'], test_data_salida['y']]).T
rmse = np.sqrt(mean_squared_error(comb_real, comb_pred))
r2 = r2_score(comb_real, comb_pred)
print("RMSE combinado:", rmse)
print("R² combinado:", r2)
mape = np.mean(np.abs((comb_real - comb_pred)/np.maximum(comb_real,1)))
print("MAPE:", mape)

### Prueba con todas las estaciones

In [ ]:
estaciones

In [ ]:
import time
best_rmse = None
best_r2 = None
best_station = None
mape_arr = []
for est in estaciones:

    # SALIDA
    station_data = resultado_fecha_junta[resultado_fecha_junta['stationID'] == est]
    #station_data = station_data.drop(columns=['stationID', 'entrada','temp','humidity','precip','windspeed','lineID'])
    station_data.rename(columns={'time': 'ds', 'salida': 'y'},inplace=True)

    train_size = int(len(station_data) * 0.8)
    train_data_salida = station_data.iloc[:train_size]
    test_data_salida = station_data.iloc[train_size:]
    # Crear un modelo Prophet
    model = Prophet()
    model.random_state = 42  # Reemplaza 42 con la semilla que desees
    model.add_regressor('lineID')
    model.add_regressor('entrada')
    model.add_regressor('temp')
    model.add_regressor('humidity')
    model.add_regressor('windspeed')

    # Ajustar el modelo a los datos
    model.fit(train_data_salida)
    # Realizar las predicciones
    forecast_salida = model.predict(test_data_salida)
    rmse_salida = np.sqrt(mean_squared_error(test_data_salida['y'], forecast_salida['yhat']))
    r2_salida = r2_score(test_data_salida['y'],forecast_salida['yhat'])




    # ENTRADA
    station_data = resultado_fecha_junta[resultado_fecha_junta['stationID'] == 0]
    #station_data = station_data.drop(columns=['stationID', 'entrada','temp','humidity','precip','windspeed','lineID'])
    station_data.rename(columns={'time': 'ds', 'entrada': 'y'},inplace=True)

    train_size = int(len(station_data) * 0.8)
    train_data_entrada = station_data.iloc[:train_size]
    test_data_entrada = station_data.iloc[train_size:]
    # Crear un modelo Prophet
    model = Prophet()
    model.random_state = 42  # Reemplaza 42 con la semilla que desees
    model.add_regressor('lineID')
    model.add_regressor('salida')
    model.add_regressor('temp')
    model.add_regressor('humidity')
    model.add_regressor('windspeed')

    # Ajustar el modelo a los datos
    model.fit(train_data_entrada)
    # Realizar las predicciones
    forecast_entrada = model.predict(test_data_entrada)
    rmse_entrada = np.sqrt(mean_squared_error(test_data_entrada['y'], forecast_entrada['yhat']))
    r2_entrada = r2_score(test_data_entrada['y'],forecast_entrada['yhat'])

    print(forecast_entrada['yhat'].shape)
    print(forecast_salida['yhat'].shape)
    comb_pred = np.array([forecast_entrada['yhat'], forecast_salida['yhat']]).T
    comb_real = np.array([test_data_entrada['y'], test_data_salida['y']]).T

    rmse = np.sqrt(mean_squared_error(comb_real, comb_pred))
    r2 = r2_score(comb_real, comb_pred)

    print('RMSE',rmse)

    #Calcula el MAPE
    mape = np.mean(np.abs((comb_real - comb_pred)/np.maximum(comb_real,1)))*100
    mape_arr.append(mape)

    if(best_rmse is None or rmse < best_rmse):
        best_rmse = rmse
        best_r2 = r2
        best_station = est
    
    time.sleep(1)

mape_total = np.mean(mape_arr)
mape_total

### 10 min Todas Estaciones con METEO

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from prophet import Prophet
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
all_stations_data = resultado_fecha_junta.copy()
all_stations_data.rename(columns={'time': 'ds', 'salida': 'y'},inplace=True)
all_stations_data

In [ ]:
train_size = int(len(all_stations_data) * 0.8)
train_data = all_stations_data.iloc[:train_size]
test_data = all_stations_data.iloc[train_size:]
# Crear un modelo Prophet
model = Prophet()
model.random_state = 42  # Reemplaza 42 con la semilla que desees
model.add_regressor('stationID')
model.add_regressor('lineID')
model.add_regressor('entrada')
model.add_regressor('temp')
model.add_regressor('humidity')
model.add_regressor('precip')
model.add_regressor('windspeed')

# Ajustar el modelo a los datos
model.fit(train_data)
# Realizar las predicciones
forecast = model.predict(test_data)
# Visualizar las predicciones
fig = model.plot(forecast)

In [ ]:
rmse_salida = np.sqrt(mean_squared_error(test_data['y'], forecast['yhat']))
print(f'RMSE: {rmse_salida}')
r2_salida = r2_score(test_data['y'],forecast['yhat'])
print(f'R^2: {r2_salida}')

### 10 min Todas Estaciones sin METEO

In [ ]:
all_stations_data = resultado_fecha_junta.copy()
all_stations_data.rename(columns={'time': 'ds', 'salida': 'y'},inplace=True)
all_stations_data.drop(columns=['stationID', 'entrada','temp','humidity','precip','windspeed','lineID'],inplace=True)
all_stations_data

In [ ]:
train_size = int(len(all_stations_data) * 0.8)
train_data = all_stations_data.iloc[:train_size]
test_data = all_stations_data.iloc[train_size:]
# Crear un modelo Prophet
model = Prophet()
model.random_state = 42  # Reemplaza 42 con la semilla que desees
# model.add_regressor('stationID')
# model.add_regressor('lineID')
# model.add_regressor('entrada')
# model.add_regressor('temp')
# model.add_regressor('humidity')
# model.add_regressor('precip')
# model.add_regressor('windspeed')

# Ajustar el modelo a los datos
model.fit(train_data)
# Realizar las predicciones
forecast = model.predict(test_data)
# Visualizar las predicciones
fig = model.plot(forecast)

In [ ]:
rmse_salida = np.sqrt(mean_squared_error(test_data['y'], forecast['yhat']))
print(f'RMSE: {rmse_salida}')
r2_salida = r2_score(test_data['y'],forecast['yhat'])
print(f'R^2: {r2_salida}')

### 1 Hora Todas Estaciones con METEO

In [ ]:
resultado_fecha_junta_hora

In [ ]:
all_stations_data = resultado_fecha_junta_hora.copy()
all_stations_data.rename(columns={'time': 'ds', 'salida': 'y'},inplace=True)
all_stations_data

In [ ]:
train_size = int(len(all_stations_data) * 0.8)
train_data = all_stations_data.iloc[:train_size]
test_data = all_stations_data.iloc[train_size:]
# Crear un modelo Prophet
model = Prophet()
model.random_state = 42  # Reemplaza 42 con la semilla que desees
model.add_regressor('stationID')
model.add_regressor('lineID')
model.add_regressor('entrada')
model.add_regressor('temp')
model.add_regressor('humidity')
model.add_regressor('precip')
model.add_regressor('windspeed')

# Ajustar el modelo a los datos
model.fit(train_data)
# Realizar las predicciones
forecast = model.predict(test_data)
# Visualizar las predicciones
fig = model.plot(forecast)

In [ ]:
rmse_salida = np.sqrt(mean_squared_error(test_data['y'], forecast['yhat']))
print(f'RMSE: {rmse_salida}')
r2_salida = r2_score(test_data['y'],forecast['yhat'])
print(f'R^2: {r2_salida}')

### 1 Hora Todas Estaciones sin METEO

In [ ]:
all_stations_data = resultado_fecha_junta_hora.copy()
all_stations_data.rename(columns={'time': 'ds', 'salida': 'y'},inplace=True)
all_stations_data.drop(columns=['temp','humidity','precip','windspeed'],inplace=True)
all_stations_data

In [ ]:
train_size = int(len(all_stations_data) * 0.8)
train_data = all_stations_data.iloc[:train_size]
test_data = all_stations_data.iloc[train_size:]
# Crear un modelo Prophet
model = Prophet()
model.random_state = 42  # Reemplaza 42 con la semilla que desees
model.add_regressor('stationID')
model.add_regressor('lineID')
model.add_regressor('entrada')
# model.add_regressor('temp')
# model.add_regressor('humidity')
# model.add_regressor('precip')
# model.add_regressor('windspeed')

# Ajustar el modelo a los datos
model.fit(train_data)
# Realizar las predicciones
forecast = model.predict(test_data)
# Visualizar las predicciones
fig = model.plot(forecast)

In [ ]:
rmse_salida = np.sqrt(mean_squared_error(test_data['y'], forecast['yhat']))
print(f'RMSE: {rmse_salida}')
r2_salida = r2_score(test_data['y'],forecast['yhat'])
print(f'R^2: {r2_salida}')

## RANDOM FOREST

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

In [ ]:
rnn_df = records.set_index('time')
rnn_df = rnn_df.groupby(['stationID', pd.Grouper(freq='H', level=0), 'status'])['userID'].count().unstack(fill_value=0)
rnn_df.rename(columns={0: 'salida', 1: 'entrada'}, inplace=True)
rnn_df.reset_index(inplace=True)
rnn_df

In [ ]:
rnn_df['year'] = rnn_df['time'].dt.year
rnn_df['month'] = rnn_df['time'].dt.month
rnn_df['day'] = rnn_df['time'].dt.day
rnn_df['hour'] = rnn_df['time'].dt.hour
rnn_df.drop(columns=['time'], inplace=True)
rnn_df

In [ ]:
# Aquí, seleccionaremos las características (estación y hora) y el objetivo (salida y entrada)
X = rnn_df[['stationID', 'year', 'month', 'day', 'hour']].values
y_salida = rnn_df['salida'].values
y_entrada = rnn_df['entrada'].values

In [ ]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_salida_train, y_salida_test, y_entrada_train, y_entrada_test = train_test_split(X, y_salida, y_entrada, test_size=0.2, random_state=42)

In [ ]:
# Crear y entrenar el modelo Random Forest para la variable "salida"
rf_salida = RandomForestRegressor(n_estimators=100, random_state=42)
rf_salida.fit(X_train, y_salida_train)

In [ ]:
# Realizar predicciones para la variable "salida"
y_salida_pred = rf_salida.predict(X_test)

In [ ]:
# Evaluar el modelo
rmse_salida = np.sqrt(mean_squared_error(y_salida_test, y_salida_pred))
r2_salida = r2_score(y_salida_test, y_salida_pred)

print(f"Root Mean Squared Error (RMSE) para la variable 'salida': {rmse_salida}")
print(f"Coeficiente de determinación (R^2) para la variable 'salida': {r2_salida}")

In [ ]:
# Crear y entrenar el modelo Random Forest para la variable "entrada"
rf_entrada = RandomForestRegressor(n_estimators=100, random_state=42)
rf_entrada.fit(X_train, y_entrada_train)

# Realizar predicciones para la variable "entrada"
y_entrada_pred = rf_entrada.predict(X_test)

# Evaluar el modelo
rmse_entrada = np.sqrt(mean_squared_error(y_entrada_test, y_entrada_pred))
r2_entrada = r2_score(y_entrada_test, y_entrada_pred)

print(f"Root Mean Squared Error (RMSE) para la variable 'entrada': {rmse_entrada}")
print(f"Coeficiente de determinación (R^2) para la variable 'entrada': {r2_entrada}")

In [ ]:
X_test

In [ ]:
# Gráfico de valores reales y predicciones para la columna "salida"
plt.figure(figsize=(12, 6))
plt.plot(y_salida_test, label='Valores Reales (Salida)', marker='o')
plt.plot(y_salida_pred, label='Predicciones (Salida)', linestyle='--', marker='x')

# Configuración de etiquetas y leyenda
plt.xlabel('Fecha y Hora')
plt.ylabel('Valor')
plt.title('Valores Reales vs. Predicciones (Salida)')
plt.legend()

# Mostrar el gráfico
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Gráfico para la variable "salida"
plt.figure(figsize=(12, 6))
plt.scatter(y_salida_test, y_salida_pred, alpha=0.5)
plt.title('Predicciones vs. Valores Reales (Salida)')
plt.xlabel('Valores Reales')
plt.ylabel('Predicciones')
plt.grid(True)
plt.show()

# Gráfico para la variable "entrada"
plt.figure(figsize=(12, 6))
plt.scatter(y_entrada_test, y_entrada_pred, alpha=0.5)
plt.title('Predicciones vs. Valores Reales (Entrada)')
plt.xlabel('Valores Reales')
plt.ylabel('Predicciones')
plt.grid(True)
plt.show()

In [ ]:
y_entrada_pred

In [ ]:
resultado

### Cada 10 min Meteo incluido

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

In [ ]:
resultado

In [ ]:
X = resultado[['stationID', 'year', 'month', 'day', 'hour','min', 'lineID', 'temp', 'humidity', 'precip', 'windspeed']].values
y_salida = resultado['salida'].values
y_entrada = resultado['entrada'].values
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_salida_train, y_salida_test, y_entrada_train, y_entrada_test = train_test_split(X, y_salida, y_entrada, test_size=0.2, random_state=42)

### Optimización de parámetros

In [ ]:
rf_salida = RandomForestRegressor()
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20]
}

# Crear y entrenar el modelo Random Forest para la variable "salida"
grid_search = GridSearchCV(rf_salida, param_grid, cv=5, scoring='neg_root_mean_squared_error', n_jobs=6)
grid_search.fit(X_train, y_salida_train)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

y_salida_pred = best_model.predict(X_test)

rmse_salida = np.sqrt(mean_squared_error(y_salida_test, y_salida_pred))
r2_salida = r2_score(y_salida_test, y_salida_pred)

In [ ]:
print(f"Root Mean Squared Error (RMSE) para la variable 'salida': {rmse_salida}")
print(f"Coeficiente de determinación (R^2) para la variable 'salida': {r2_salida}")

In [ ]:
best_params

In [ ]:
#Guardar mejor modelo
import joblib
joblib.dump(best_model, 'best_random_forest_model.pkl')

### Modelos con mejores parámetros

In [ ]:
# Crear y entrenar el modelo Random Forest para la variable "salida"
rf_salida = RandomForestRegressor(n_estimators=300, max_depth=20,random_state=42)
rf_salida.fit(X_train, y_salida_train)

# Realizar predicciones para la variable "salida"
y_salida_pred = rf_salida.predict(X_test)

# Evaluar el modelo
rmse_salida = np.sqrt(mean_squared_error(y_salida_test, y_salida_pred))
r2_salida = r2_score(y_salida_test, y_salida_pred)

print(f"Root Mean Squared Error (RMSE) para la variable 'salida': {rmse_salida}")
print(f"Coeficiente de determinación (R^2) para la variable 'salida': {r2_salida}")

In [ ]:
# Crear y entrenar el modelo Random Forest para la variable "entrada"
rf_entrada = RandomForestRegressor(n_estimators=300, max_depth=20,random_state=42)
rf_entrada.fit(X_train, y_entrada_train)

# Realizar predicciones para la variable "entrada"
y_entrada_pred = rf_entrada.predict(X_test)

# Evaluar el modelo
rmse_entrada = np.sqrt(mean_squared_error(y_entrada_test, y_entrada_pred))
r2_entrada = r2_score(y_entrada_test, y_entrada_pred)

print(f"Root Mean Squared Error (RMSE) para la variable 'entrada': {rmse_entrada}")
print(f"Coeficiente de determinación (R^2) para la variable 'entrada': {r2_entrada}")

### Cada 1 hora con datos Meteo

In [ ]:
resultado_hora

In [ ]:
X = resultado_hora[['stationID', 'year', 'month', 'day', 'hour', 'lineID', 'temp', 'humidity', 'precip', 'windspeed']].values
y_salida = resultado_hora['salida'].values
y_entrada = resultado_hora['entrada'].values
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_salida_train, y_salida_test, y_entrada_train, y_entrada_test = train_test_split(X, y_salida, y_entrada, test_size=0.2, random_state=42)

In [ ]:
# Crear y entrenar el modelo Random Forest para la variable "salida"
rf_salida = RandomForestRegressor(n_estimators=300, max_depth=20,random_state=42)
rf_salida.fit(X_train, y_salida_train)

# Realizar predicciones para la variable "salida"
y_salida_pred = rf_salida.predict(X_test)

# Evaluar el modelo
rmse_salida = np.sqrt(mean_squared_error(y_salida_test, y_salida_pred))
r2_salida = r2_score(y_salida_test, y_salida_pred)

print(f"Root Mean Squared Error (RMSE) para la variable 'salida': {rmse_salida}")
print(f"Coeficiente de determinación (R^2) para la variable 'salida': {r2_salida}")

In [ ]:
# Crear y entrenar el modelo Random Forest para la variable "entrada"
rf_entrada = RandomForestRegressor(n_estimators=300, max_depth=20,random_state=42)
rf_entrada.fit(X_train, y_entrada_train)

# Realizar predicciones para la variable "entrada"
y_entrada_pred = rf_entrada.predict(X_test)

# Evaluar el modelo
rmse_entrada = np.sqrt(mean_squared_error(y_entrada_test, y_entrada_pred))
r2_entrada = r2_score(y_entrada_test, y_entrada_pred)

print(f"Root Mean Squared Error (RMSE) para la variable 'entrada': {rmse_entrada}")
print(f"Coeficiente de determinación (R^2) para la variable 'entrada': {r2_entrada}")

## XGBOOST

In [ ]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
import joblib
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

### 10 min Estación única sin METEO

In [ ]:
resultado

In [ ]:
X = resultado_hora[['stationID', 'year', 'month', 'day', 'hour', 'lineID', 'temp', 'humidity', 'precip', 'windspeed']].values
y_salida = resultado_hora['salida'].values
y_entrada = resultado_hora['entrada'].values
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_salida_train, y_salida_test, y_entrada_train, y_entrada_test = train_test_split(X, y_salida, y_entrada, test_size=0.2, random_state=42)

In [ ]:
train_size = int(len(resultado) * 0.8)
train_data = resultado.iloc[:train_size]
test_data = resultado.iloc[train_size:]

In [ ]:
train_data

In [ ]:
test_data

In [ ]:
X_train = train_data[['stationID', 'year', 'month', 'day', 'hour', 'min', 'lineID', 'temp', 'humidity', 'precip', 'windspeed']].values
X_test = test_data[['stationID', 'year', 'month', 'day', 'hour', 'min', 'lineID', 'temp', 'humidity', 'precip', 'windspeed']].values
y_salida_train = train_data['salida'].values
y_salida_test = test_data['salida'].values
y_entrada_train = train_data['entrada'].values
y_entrada_test = test_data['entrada'].values

In [ ]:
y_salida_test

In [ ]:
xgb_salida = xgb.XGBRegressor(n_estimators = 300, learning_rate = 0.1, max_depth = 12, random_state=42)
# Realizar predicciones para la variable "salida"
xgb_salida.fit(X_train, y_salida_train)
y_salida_pred = xgb_salida.predict(X_test)

# Evaluar el modelo para la variable "salida"
rmse_salida = np.sqrt(mean_squared_error(y_salida_test, y_salida_pred))
r2_salida = r2_score(y_salida_test, y_salida_pred)

print(f"Root Mean Squared Error (RMSE) para la variable 'salida': {rmse_salida}")
print(f"Coeficiente de determinación (R^2) para la variable 'salida': {r2_salida}")

### Testing

In [ ]:
from xgboost import XGBRegressor

In [ ]:
rnn_df = records.set_index('time')
rnn_df = rnn_df.groupby(['stationID', pd.Grouper(freq='H', level=0), 'status'])['userID'].count().unstack(fill_value=0)
rnn_df.rename(columns={0: 'salida', 1: 'entrada'}, inplace=True)
rnn_df.reset_index(inplace=True)


rnn_df['year'] = rnn_df['time'].dt.year
rnn_df['month'] = rnn_df['time'].dt.month
rnn_df['day'] = rnn_df['time'].dt.day
rnn_df['hour'] = rnn_df['time'].dt.hour
rnn_df.drop(columns=['time'], inplace=True)

# Aquí, seleccionaremos las características (estación y hora) y el objetivo (salida y entrada)
X = rnn_df[['stationID', 'year', 'month', 'day', 'hour']].values
y_salida = rnn_df['salida'].values
y_entrada = rnn_df['entrada'].values

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_salida_train, y_salida_test, y_entrada_train, y_entrada_test = train_test_split(X, y_salida, y_entrada, test_size=0.2, random_state=42)

In [ ]:
rnn_df

In [ ]:
# Crear y entrenar el modelo XGBoost para la variable "salida"
xgb_salida = XGBRegressor(n_estimators=100, random_state=42)
xgb_salida.fit(X_train, y_salida_train)

# Realizar predicciones para la variable "salida"
y_salida_pred = xgb_salida.predict(X_test)

# Evaluar el modelo para la variable "salida"
rmse_salida = np.sqrt(mean_squared_error(y_salida_test, y_salida_pred))
r2_salida = r2_score(y_salida_test, y_salida_pred)

print(f"Root Mean Squared Error (RMSE) para la variable 'salida': {rmse_salida}")
print(f"Coeficiente de determinación (R^2) para la variable 'salida': {r2_salida}")

In [ ]:
# Crear y entrenar el modelo XGBoost para la variable "entrada"
xgb_entrada = XGBRegressor(n_estimators=100, random_state=42)
xgb_entrada.fit(X_train, y_entrada_train)

# Realizar predicciones para la variable "entrada"
y_entrada_pred = xgb_entrada.predict(X_test)

# Evaluar el modelo para la variable "entrada"
rmse_entrada = np.sqrt(mean_squared_error(y_entrada_test, y_entrada_pred))
r2_entrada = r2_score(y_entrada_test, y_entrada_pred)

print(f"Root Mean Squared Error (RMSE) para la variable 'entrada': {rmse_entrada}")
print(f"Coeficiente de determinación (R^2) para la variable 'entrada': {r2_entrada}")

In [ ]:
# Gráfico para la variable "salida"
plt.figure(figsize=(12, 6))
plt.scatter(y_salida_test, y_salida_pred, alpha=0.5)
plt.title('Predicciones vs. Valores Reales (Salida)')
plt.xlabel('Valores Reales')
plt.ylabel('Predicciones')
plt.grid(True)
plt.show()

# Gráfico para la variable "entrada"
plt.figure(figsize=(12, 6))
plt.scatter(y_entrada_test, y_entrada_pred, alpha=0.5)
plt.title('Predicciones vs. Valores Reales (Entrada)')
plt.xlabel('Valores Reales')
plt.ylabel('Predicciones')
plt.grid(True)
plt.show()

### Cada 10 min y Meteo incluido

In [ ]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
import joblib
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

In [ ]:
resultado

In [ ]:
X = resultado[['stationID', 'year', 'month', 'day', 'hour', 'min', 'lineID', 'temp', 'humidity', 'precip', 'windspeed']].values
y_salida = resultado['salida'].values
y_entrada = resultado['entrada'].values
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_salida_train, y_salida_test, y_entrada_train, y_entrada_test = train_test_split(X, y_salida, y_entrada, test_size=0.2, random_state=42)

### Optimización de parámetros

In [ ]:
model = xgb.XGBRegressor(random_state=42)
param_grid = {
    'n_estimators': [300, 400, 500],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [10, 12, 15]
}

grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=6)
grid_search.fit(X_train, y_salida_train)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

y_pred = best_model.predict(X_test)

In [ ]:
# Evaluar el modelo para la variable "salida"
rmse_salida = np.sqrt(mean_squared_error(y_salida_test, y_pred))
r2_salida = r2_score(y_salida_test, y_pred)

print(f"Root Mean Squared Error (RMSE) para la variable 'salida': {rmse_salida}")
print(f"Coeficiente de determinación (R^2) para la variable 'salida': {r2_salida}")

In [ ]:
import joblib
joblib.dump(best_model, 'best_xgboost_model_salida_10.pkl')

In [ ]:
best_params

In [ ]:
model = xgb.XGBRegressor(random_state=42)
param_grid = {
    'n_estimators': [300, 400, 500],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [10, 12, 15]
}

grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=6)
grid_search.fit(X_train, y_entrada_train)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

y_pred = best_model.predict(X_test)

In [ ]:
# Evaluar el modelo para la variable "salida"
rmse_salida = np.sqrt(mean_squared_error(y_entrada_test, y_pred))
r2_salida = r2_score(y_entrada_test, y_pred)

print(f"Root Mean Squared Error (RMSE) para la variable 'salida': {rmse_salida}")
print(f"Coeficiente de determinación (R^2) para la variable 'salida': {r2_salida}")

In [ ]:
joblib.dump(best_model, 'best_xgboost_model_entrada_10.pkl')

In [ ]:
best_params

### Usamos los mejores parámetros

In [ ]:
resultado

In [ ]:
X = resultado[['stationID', 'year', 'month', 'day', 'hour', 'min', 'lineID', 'temp', 'humidity', 'precip', 'windspeed']].values
y_salida = resultado['salida'].values
y_entrada = resultado['entrada'].values
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_salida_train, y_salida_test, y_entrada_train, y_entrada_test = train_test_split(X, y_salida, y_entrada, test_size=0.2, random_state=42)

In [ ]:
xgb_salida = xgb.XGBRegressor(n_estimators = 300, learning_rate = 0.1, max_depth = 12, random_state=42)
# Realizar predicciones para la variable "entrada"
xgb_salida.fit(X_train, y_salida_train)
y_salida_pred = xgb_salida.predict(X_test)

# Evaluar el modelo para la variable "entrada"
rmse_salida = np.sqrt(mean_squared_error(y_salida_test, y_salida_pred))
r2_salida = r2_score(y_salida_test, y_salida_pred)

print(f"Root Mean Squared Error (RMSE) para la variable 'salida': {rmse_entrada}")
print(f"Coeficiente de determinación (R^2) para la variable 'salida': {r2_entrada}")

In [ ]:
xgb_entrada = xgb.XGBRegressor(n_estimators = 300, learning_rate = 0.1, max_depth = 12, random_state=42)
# Realizar predicciones para la variable "entrada"
xgb_entrada.fit(X_train, y_entrada_train)
y_entrada_pred = xgb_entrada.predict(X_test)

# Evaluar el modelo para la variable "entrada"
rmse_entrada = np.sqrt(mean_squared_error(y_entrada_test, y_entrada_pred))
r2_entrada = r2_score(y_entrada_test, y_entrada_pred)

print(f"Root Mean Squared Error (RMSE) para la variable 'entrada': {rmse_entrada}")
print(f"Coeficiente de determinación (R^2) para la variable 'entrada': {r2_entrada}")

In [ ]:
joblib.dump(xgb_entrada, 'best_xgboost_model_entrada.pkl')

In [ ]:
# Gráfico para la variable "salida"
plt.figure(figsize=(12, 6))
plt.scatter(y_salida_test, y_salida_pred, alpha=0.5)
plt.title('Predicciones vs. Valores Reales (Salida)')
plt.xlabel('Valores Reales')
plt.ylabel('Predicciones')
plt.grid(True)
plt.show()

# Gráfico para la variable "entrada"
plt.figure(figsize=(12, 6))
plt.scatter(y_entrada_test, y_entrada_pred, alpha=0.5)
plt.title('Predicciones vs. Valores Reales (Entrada)')
plt.xlabel('Valores Reales')
plt.ylabel('Predicciones')
plt.grid(True)
plt.show()

In [ ]:
# Gráfico de valores reales y predicciones para la columna "salida"
plt.figure(figsize=(12, 6))
plt.plot(y_entrada_test, label='Valores Reales (Entrada)', marker='o')
plt.plot(y_entrada_pred, label='Predicciones (Entrada)', linestyle='--', marker='x')

# Configuración de etiquetas y leyenda
plt.xlabel('Fecha y Hora')
plt.ylabel('Valor')
plt.title('Valores Reales vs. Predicciones (Entrada)')
plt.legend()

# Mostrar el gráfico
plt.grid(True)
plt.tight_layout()
plt.show()

### Cada 1 hora con datos Meteo

In [ ]:
resultado_hora

In [ ]:
X = resultado_hora[['stationID', 'year', 'month', 'day', 'hour', 'lineID', 'temp', 'humidity', 'precip', 'windspeed']].values
y_salida = resultado_hora['salida'].values
y_entrada = resultado_hora['entrada'].values
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_salida_train, y_salida_test, y_entrada_train, y_entrada_test = train_test_split(X, y_salida, y_entrada, test_size=0.2, random_state=42)

In [ ]:
xgb_salida = xgb.XGBRegressor(n_estimators = 300, learning_rate = 0.1, max_depth = 12, random_state=42)
# Realizar predicciones para la variable "salida"
xgb_salida.fit(X_train, y_salida_train)
y_salida_pred = xgb_salida.predict(X_test)

# Evaluar el modelo para la variable "salida"
rmse_salida = np.sqrt(mean_squared_error(y_salida_test, y_salida_pred))
r2_salida = r2_score(y_salida_test, y_salida_pred)

print(f"Root Mean Squared Error (RMSE) para la variable 'salida': {rmse_salida}")
print(f"Coeficiente de determinación (R^2) para la variable 'salida': {r2_salida}")

In [ ]:
xgb_entrada = xgb.XGBRegressor(n_estimators = 300, learning_rate = 0.1, max_depth = 12, random_state=42)
# Realizar predicciones para la variable "entrada"
xgb_entrada.fit(X_train, y_entrada_train)
y_entrada_pred = xgb_entrada.predict(X_test)

# Evaluar el modelo para la variable "entrada"
rmse_entrada = np.sqrt(mean_squared_error(y_entrada_test, y_entrada_pred))
r2_entrada = r2_score(y_entrada_test, y_entrada_pred)

print(f"Root Mean Squared Error (RMSE) para la variable 'entrada': {rmse_entrada}")
print(f"Coeficiente de determinación (R^2) para la variable 'entrada': {r2_entrada}")

In [ ]:
# Gráfico para la variable "entrada"
plt.figure(figsize=(12, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.title('Predicciones vs. Valores Reales (Entrada)')
plt.xlabel('Valores Reales')
plt.ylabel('Predicciones')
plt.grid(True)
plt.show()